In [1]:
from elasticsearch import Elasticsearch
from konlpy.tag import Okt

import pandas as pd
import matplotlib
matplotlib.use("TkAgg")
import matplotlib.pyplot as plt

from wordcloud import WordCloud
from collections import Counter

import re

In [2]:
spliter = Okt()
keeplooping = True
Noun = ""
currentCount = 0

In [3]:
stopword = ['뉴시스','무단','이데일리''헤럴드','한국','포럼','노컷','중앙','이데일리','종목','우리나라',\
            '전자','라이프','누구','가운데','서울','경향','매일','한경닷컴','경제','라이온',\
            '오마이','아이','연합','지난해','지난달','때문','저작권','생방송','내용','네이버','영상','청춘','우스']

In [4]:
while keeplooping:
    es_client = Elasticsearch(hosts=[{'host': '52.141.39.217', 'port': 5000}])
    query = {
      "_source": ["ai_keyword"],
      "from": currentCount,
      "size": 10000,
      "sort": [
        {
          "reg_date": {
            "order": "desc"
          }
        }
      ],
      "query": {
        "bool": {
          "must": [
            {
              "term": {
                "is_ai_summary": "1"
              }
            },
            {
              "match": {
                "language": "ko"
              }
            },
            {
              "range": {
                "reg_date": {
                  "gte": "2019-04-01", 
                  "lte": "2019-05-09"
                }
              }
            }
          ]
        }
      }
    }
    res = es_client.search(index='news', doc_type='data', body=query)
    
    if len(res["hits"]["hits"]) == 0:
        keeplooping = False

    for ress in res["hits"]["hits"]:
        for resss in ress["_source"]["ai_keyword"]:
            tmps = spliter.pos(resss)
            for tmp in tmps:
                if tmp[1] == "Noun" and tmp[0] not in stopword:
                        if re.search('뉴스',tmp[0]) or re.search('신문',tmp[0]) or re.search('칼럼',tmp[0]) or re.search('일보',tmp[0])\
                            or re.search('대한민국',tmp[0]) or re.search('보기',tmp[0]) or re.search('경제',tmp[0]):
                            continue
                        Noun += "," + tmp[0]
                    
    currentCount += 10000

In [ ]:
wordcloud = WordCloud(\
                     font_path = 'C:/Windows/Fonts/malgun.ttf',\
                      background_color='white',\
                      max_words=200,\
                      max_font_size=40,\
                      random_state=42\
                     ).generate(Noun)

fig = plt.figure(figsize=(12,12))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()
# fig.savefig("word-en.png", dpi=900)
fig.savefig("word-ko.png")

print(wordcloud.words_)